In [6]:
%load_ext autoreload
%autoreload 1 
%aimport game

from game import Game
from team import Team
from utility import headers

from collections import defaultdict
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from itertools import combinations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
g = Game(3970331,source='file')

In [217]:
base_schedule = "https://stats.ncaa.org/season_divisions/18220/livestream_scoreboards?utf8=%E2%9C%93&season_division_id="

In [3]:
d1Teams = pd.read_csv("./Data/ncaa_synergy_id_names_map_2023_2024.csv").ncaaName.tolist()
'UConn' in d1Teams

True

In [538]:
seasonStartDate = datetime.date(2023, 11, 6)
today = datetime.date(2023, 11, 9)#datetime.date.today()
date_list = [seasonStartDate + datetime.timedelta(days=x) for x in range( (today - seasonStartDate).days +1 )]

In [196]:
r = requests.get(base_schedule, headers=headers)

In [197]:
soup = BeautifulSoup(r.text)

In [7]:
x = pd.read_csv("games_2023_2024.csv", parse_dates=['date'])
x.date = x.date.dt.date

In [539]:
gameIds = [tag['href'] for tag in soup.find_all('a', string='Box Score')]
gameIds = [link.split('/')[2] for link in gameIds]

In [540]:
date = date_list[-1]

In [541]:
date

datetime.date(2023, 11, 9)

In [551]:
r = requests.get(base_schedule + f"&game_date={date.month}%2F{date.day}%2F{date.year}", headers=headers)
    
if r.status_code != 200:
    raise Exception(f"Request for {date} was not successful")
else:
    soup = BeautifulSoup(r.text)
    teamTags = soup.find_all('td', attrs={'class':'opponents_min_width'})
    gameIds = [tag['href'] for tag in soup.find_all('a', string='Box Score')]
    gameIds = [link.split('/')[2] for link in gameIds]

    cancelledGames = soup.find_all('div',attrs={'class':'table-responsive'})
    cancelledGames = [ix for ix, game in enumerate(cancelledGames) if 'Canceled' in game.text]

    names1 = []
    names2 = []
    ids1 = []
    ids2 = []

    for i, tag in enumerate(teamTags):
        name = tag.text.strip()
        
        
        if len(tag.contents) > 1:
            teamid = tag.contents[1]['href']
            teamid = teamid.split('/')[-1]
            name = "(".join(name.split(' (')[:-1]) # Get rid of record attached to team name
        else:
            teamid = None
        
        if i % 2 == 0:
            names1.append(name)
            ids1.append(teamid)
        else:
            names2.append(name)
            ids2.append(teamid)

    for cancelled in cancelledGames:
        names1.pop(cancelled)
        ids1.pop(cancelled)
        names2.pop(cancelled)
        ids2.pop(cancelled)
    
    schedule = pd.DataFrame({'teamName1':names1, 'teamId1':ids1, 'teamName2':names2, 'teamId2':ids2, 'gameId': gameIds})
    schedule['date'] = seasonStartDate

In [543]:
a = soup.find_all('div',attrs={'class':'table-responsive'})

In [546]:
[ix for ix, game in enumerate(a) if 'Canceled' in game.text]

[14]

In [116]:
schedule.to_csv('games_2023_2024.csv', index=False)

In [301]:
games = []
for row in tqdm(schedule.itertuples()):
    if row.teamName1 in d1Teams and row.teamName2 in d1Teams:
        games.append(Game(row.gameId, source='file'))

30it [00:27,  1.09it/s]


In [240]:
playerBoxScores = pd.DataFrame()

for game in tqdm(games):
    for teamid, box in game.boxScore.items():
        stats = box.iloc[:-2].copy()
        stats['Team ID'] = teamid
        stats['Opponent ID'] = [team for team in game.teams if team != teamid][0]
        stats['Game ID'] = game.gameId
        playerBoxScores = pd.concat([playerBoxScores, stats])

100%|██████████| 25/25 [00:00<00:00, 424.88it/s]


In [241]:
playerBoxScores

,#,Name,P,MP,GS,FGM,FGA,3FG,3FGA,FT,...,AST,TO,STL,BLK,PF,DQ,Tech Fouls,Team ID,Opponent ID,Game ID
0,20.0,Brooke Berry,G,6:01,0,0,3,0,1,0,...,0,1,0,0,0,0,0,560912,560627,3970332
1,13.0,Drea Brumfield,G,2:53,0,0,0,0,0,0,...,0,0,0,1,0,0,0,560912,560627,3970332
2,15.0,Isobel Bunyan,G,24:13,0,7,12,1,3,0,...,2,2,3,0,1,0,0,560912,560627,3970332
3,21.0,Lexi Deden,F,25:06,1,1,8,0,0,5,...,0,3,1,2,2,0,0,560912,560627,3970332
4,32.0,Marah Dykstra,F,24:07,1,2,9,0,2,2,...,4,1,0,1,4,0,0,560912,560627,3970332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,14.0,Marina Hernandez,F,2:40,0,0,0,0,0,0,...,0,0,0,0,0,0,0,561073,560653,3970347
4,20.0,Aya Kéita,G,22:51,0,2,6,2,5,0,...,1,1,3,0,2,0,0,561073,560653,3970347
5,1.0,Luana Leite,G,28:16,1,1,5,0,0,2,...,2,5,1,0,3,0,0,561073,560653,3970347
6,22.0,Paula Tirado,G,16:36,0,2,4,2,3,0,...,1,1,0,0,4,0,0,561073,560653,3970347


In [302]:
teamBoxScores = pd.DataFrame()

for game in tqdm(games):
    for teamid, box in game.boxScore.items():
        stats = box.iloc[-1].copy()
        stats['teamId'] = teamid
        stats['opponentId'] = [team for team in game.teams if team != teamid][0]
        stats['gameId'] = game.gameId
        teamBoxScores = pd.concat([teamBoxScores, stats.to_frame().T])
teamBoxScores = teamBoxScores.drop(columns=['#', 'P'])
teamBoxScores = teamBoxScores.reset_index(drop=True)
    

100%|██████████| 25/25 [00:00<00:00, 85.14it/s]


In [338]:
teamBoxScores.head()

,Name,MP,GS,FGM,FGA,3FG,3FGA,FT,FTA,PTS,...,AST,TO,STL,BLK,PF,DQ,Tech Fouls,teamId,opponentId,gameId
0,Montana St.,200:00,1,20,61,6,19,14,21,60,...,14,12,13,4,23,0,0,560912,560627,3970332
1,BYU,200:00,1,21,49,8,21,18,28,68,...,10,22,10,2,18,0,0,560627,560912,3970332
2,Penn St.,200:00,1,35,59,12,25,12,17,94,...,22,19,15,6,16,0,0,561013,560634,3954316
3,Bucknell,200:00,1,17,48,6,19,11,18,51,...,11,26,10,2,13,0,0,560634,561013,3954316
4,Bowling Green,200:00,1,33,59,12,21,11,23,89,...,21,15,7,3,23,0,0,560623,560681,3970327


In [118]:
games[7].gameId

'3970331'

In [2]:
g = Game(3970331)

In [342]:
schedule.columns

Index(['teamName1', 'teamId1', 'teamName2', 'teamId2', 'gameId', 'date'], dtype='object')

In [340]:
pd.DataFrame(columns=['Name', 'MP', 'GS', 'FGM', 'FGA', '3FG', '3FGA', 'FT', 'FTA',
       'PTS', 'ORebs', 'DRebs', 'Tot Reb', 'AST', 'TO', 'STL', 'BLK', 'PF',
       'DQ', 'Tech Fouls', 'Team ID', 'Opponent ID', 'Game ID'])

,Name,MP,GS,FGM,FGA,3FG,3FGA,FT,FTA,PTS,...,AST,TO,STL,BLK,PF,DQ,Tech Fouls,Team ID,Opponent ID,Game ID


In [447]:
lineupData = defaultdict(lambda: defaultdict(int))


for g in games:
    lineupData = defaultdict(lambda: defaultdict(int))
    ends = np.where([item.isPossessionEnding for item in g.pbp.items])[0]+1
    poss = np.split(g.pbp.items,ends)



    for p in poss:
        # Event occurances for each team during the possession
        x = {team:[e.eventType if e.team==team else -1*e.eventType if e.team else None for e in p ] for team in g.teams}
        if any([ any(events) for events in x.values() ]):
            for team in g.teams:
                # Count the number of occurances for each event type while lineups where on the floor
                for lineup in list(combinations(p[-1].currentLineups[team],1)):
                    for pt in x[team]:
                        lineupData[lineup[0]][pt] += 1
                    lineupData[lineup[0]]['poss'] += 1
    possessions = sum([ any(e.team for e in p) for p in poss ]) / 2
    print(possessions)
    for team in g.teams:
        print(sum([v['poss'] for v in {k:v for k,v in dict(lineupData).items() if k[0] in g.boxScore[team].Name.tolist()}.values()])/10)
    print('_____')


77.0
0.0
0.0
_____
77.5
0.0
0.0
_____
73.5
0.0
0.0
_____
70.5
0.0
0.0
_____
64.0
0.0
0.0
_____
73.5
0.0
0.0
_____
77.5
0.0
0.0
_____
80.5
0.0
0.0
_____
74.0
0.0
0.0
_____
66.0
0.0
0.0
_____
78.5
0.0
0.0
_____
65.0
0.0
0.0
_____
69.5
0.0
0.0
_____
64.5
0.0
0.0
_____
73.0
0.0
0.0
_____
68.5
0.0
0.0
_____
73.0
0.0
0.0
_____
70.0
0.0
0.0
_____
72.0
0.0
0.0
_____
71.5
0.0
0.0
_____
64.5
0.0
0.0
_____
79.5
0.0
0.0
_____
93.0
0.0
0.0
_____
71.5
0.0
0.0
_____
83.5
0.0
0.0
_____


In [307]:
g.teams

['560971', '560599']

In [317]:
g=games[3]
g.gameId

'3970328'

In [318]:
ends = np.where([item.isPossessionEnding for item in g.pbp.items])[0]+1
poss = np.split(g.pbp.items,ends)
possessions = sum([ any(e.team for e in p) for p in poss ]) / 2
possessions

70.5

In [ ]:
for q in g.pbp.items:
    print(q.description)

In [322]:
g.teams

['561051', '561039']

In [336]:
lineupData = defaultdict(lambda: defaultdict(int), df.T.to_dict())#defaultdict(lambda: defaultdict(int))




for p in poss:
    # Event occurances for each team during the possession
    x = {team:[e.eventType if e.team==team else -1*e.eventType if e.team else None for e in p ] for team in g.teams}
    if any([ any(events) for events in x.values() ]):
        for team in g.teams:
            # Count the number of occurances for each event type while lineups where on the floor
            
            for lineup in list(combinations(p[-1].currentLineups[team],1)):
                for pt in x[team]:
                    lineupData[lineup][pt] += 1
                lineupData[lineup]['poss'] += 1
                
            
            if sum([v for v in {k:v['poss'] for k,v in dict(lineupData).items() if k[0] in g.boxScore[g.teams[0]].Name.tolist()}.values()])==673:
                breaka
            if len(p[-1].currentLineups['561051'])<5:
                breaka
        prev = p

        #print(len(set(p[-1].currentLineups["561039"])))

In [321]:
for e in p:
    print(e.description, e.currentLineups)

jumpball startperiod {'561051': ['Sajada Bonner', 'Sierra Johnson', "Ny'Ceara Pryor", 'Olivia Tucker', 'Kelsey Wood'], '561039': ['Dee Dee Davis', 'Teisha Hyman', 'Tenin Magassa', 'Sophie Phillips', 'Mayé Touré']}
period start {'561051': ['Sajada Bonner', 'Sierra Johnson', "Ny'Ceara Pryor", 'Olivia Tucker', 'Kelsey Wood'], '561039': ['Dee Dee Davis', 'Teisha Hyman', 'Tenin Magassa', 'Sophie Phillips', 'Mayé Touré']}
game start {'561051': ['Sajada Bonner', 'Sierra Johnson', "Ny'Ceara Pryor", 'Olivia Tucker', 'Kelsey Wood'], '561039': ['Dee Dee Davis', 'Teisha Hyman', 'Tenin Magassa', 'Sophie Phillips', 'Mayé Touré']}
Tenin Magassa, jumpball won {'561051': ['Sajada Bonner', 'Sierra Johnson', "Ny'Ceara Pryor", 'Olivia Tucker', 'Kelsey Wood'], '561039': ['Dee Dee Davis', 'Teisha Hyman', 'Tenin Magassa', 'Sophie Phillips', 'Mayé Touré']}
Kelsey Wood, jumpball lost {'561051': ['Sajada Bonner', 'Sierra Johnson', "Ny'Ceara Pryor", 'Olivia Tucker', 'Kelsey Wood'], '561039': ['Dee Dee Davis', 'T

In [325]:
playerposs = {k:v['poss'] for k,v in dict(lineupData).items() if k[0] in g.boxScore[g.teams[1]].Name.tolist()}

In [337]:
{k:v['poss'] for k,v in dict(lineupData).items()}

{('Sajada Bonner',): 130.0,
 ('Sierra Johnson',): 148.0,
 ("Ny'Ceara Pryor",): 196.0,
 ('Olivia Tucker',): 210.0,
 ('Kelsey Wood',): 94.0,
 ('Dee Dee Davis',): 164.0,
 ('Teisha Hyman',): 168.0,
 ('Tenin Magassa',): 164.0,
 ('Sophie Phillips',): 192.0,
 ('Mayé Touré',): 138.0,
 ('Ciara Brannon',): 72.0,
 ('Amelia Wood',): 196.0,
 ('Hawa Komara',): 114.0,
 ('Ines Debroise',): 140.0,
 ('Sophia Vital',): 94.0,
 ('Anaelle Dutat',): 122.0,
 ('Nalyce Dudley',): 28.0,
 ('Emma Kirby',): 116.0,
 ('Faith Pappas',): 86.0,
 ('Thai Davis',): 52.0,
 ('Eva DeChent',): 62.0,
 ('Afia Owusu-Mensah',): 62.0,
 ('Sophie Sene',): 26.0,
 ('Katie Ledden',): 26.0,
 ('Montana Hicks',): 20.0}

In [285]:
playerposs

{('Ioanna Krimili',): 110,
 ('Kemery Martin',): 108,
 ('Leilani McIntosh',): 130,
 ('Marta Suárez',): 78,
 ('McKayla Williams',): 103,
 ('Mia Mastrov',): 80,
 ('Ugonne Onyiah',): 24,
 ('Ila Lane',): 13,
 ('Alma Elsnitz',): 4}

In [345]:
d = {'a':1,'b':2,'c':3}
pd.DataFrame(d.items(), columns=)

,0,1
0,a,1
1,b,2
2,c,3


In [432]:
teams_dict = {'a':{'rating':[1, 2], 'poss':[40, 20]}, 'b':{'rating':[.9, .95], 'poss':[40, 45]}}

In [388]:
df

,teamid,adjOff,adjDef,possOff,possDef,date
0,a,1,1,40,40,1
1,b,0.9,0.9,40,40,1


In [434]:
{team['teamid']:{'rating':[team['adjOff'],team['adjDef']], 'poss':[team['possOff'],team['possDef']]} for team in df.to_dict(orient='records')}

{'a': {'rating': [1, 2], 'poss': [40, 20]},
 'b': {'rating': [0.9, 0.95], 'poss': [40, 45]}}

In [398]:
teams_dict

{'a': {'rating': [1, 1], 'poss': [40, 40]},
 'b': {'rating': [0.9, 0.9], 'poss': [40, 40]}}

[{'teamid': 'a',
  'adjOff': 1,
  'adjDef': 1,
  'possOff': 40,
  'possDef': 40,
  'date': 1},
 {'teamid': 'b',
  'adjOff': 0.9,
  'adjDef': 0.9,
  'possOff': 40,
  'possDef': 40,
  'date': 1}]

In [435]:
pd.DataFrame(teams_dict)

,a,b
rating,"[1, 2]","[0.9, 0.95]"
poss,"[40, 20]","[40, 45]"


In [465]:
df = pd.DataFrame(teams_dict)
df = df.explode(df.columns.tolist()).T
df.reset_index(inplace=True)
df['date'] = 1
df.columns = ['teamid','adjOff','adjDef','possOff','possDef','date']

In [466]:
df

,teamid,adjOff,adjDef,possOff,possDef,date
0,a,1,2,40,20,1
1,b,0.9,0.95,40,45,1


In [284]:
posscount = sum([v for v in playerposs.values()])
posscount

650

In [281]:
possessions

67.5

In [417]:
df =pd.DataFrame(columns=[ 'poss', 1,   -1,   2,  -2,   3,   -3,   4,  -4,   5,   -5,   6,  -6,   7,  -7,
         8,  -8,   9,  -9,  10,  -10,  11, -11,  12,  -12,  13, -13,  14,  -14,
        15,  -15,  16,  -16,  17, -17,  18, -18])

In [448]:
df = pd.DataFrame(dict(lineupData)).T
m = defaultdict(lambda: defaultdict(int), df.T.to_dict())

In [449]:
df.to_csv('a.csv')

In [458]:
m.rename(columns={m.columns[0]:None})

,None,1,-1,3,-9,poss,10,-10,-18,-2,...,12,-13,5,-12,-11,17,2,-14,-19,11
Debora Dos Santos,11.0,1.0,1.0,14.0,20.0,119.0,28.0,28.0,9.0,2.0,...,8.0,13.0,13.0,4.0,2.0,2.0,1.0,1.0,NaN,NaN
Cami Fulcher,9.0,1.0,1.0,16.0,23.0,129.0,25.0,25.0,8.0,5.0,...,8.0,12.0,13.0,3.0,NaN,1.0,1.0,2.0,NaN,NaN
Jasmine Gayles,11.0,1.0,1.0,18.0,30.0,162.0,35.0,35.0,10.0,5.0,...,10.0,15.0,16.0,5.0,3.0,2.0,1.0,2.0,NaN,NaN
Luana Leite,9.0,1.0,1.0,13.0,20.0,121.0,28.0,28.0,10.0,2.0,...,8.0,12.0,12.0,4.0,2.0,2.0,1.0,1.0,NaN,NaN
Freja Werth,11.0,1.0,1.0,15.0,25.0,135.0,30.0,30.0,6.0,4.0,...,6.0,10.0,12.0,4.0,2.0,2.0,1.0,2.0,NaN,NaN
Skylar Burke,9.0,1.0,1.0,14.0,19.0,118.0,23.0,23.0,9.0,1.0,...,3.0,11.0,18.0,9.0,NaN,2.0,5.0,2.0,3.0,2.0
Anya Choice,9.0,1.0,1.0,10.0,15.0,85.0,20.0,20.0,5.0,1.0,...,3.0,12.0,9.0,5.0,NaN,1.0,2.0,3.0,3.0,2.0
Flora Goed,7.0,1.0,1.0,12.0,19.0,100.0,21.0,21.0,8.0,1.0,...,3.0,14.0,13.0,5.0,NaN,2.0,4.0,3.0,1.0,1.0
Alyssa Marin,7.0,1.0,1.0,12.0,17.0,97.0,21.0,21.0,6.0,1.0,...,3.0,16.0,12.0,5.0,NaN,2.0,1.0,3.0,2.0,NaN
Alexis Whitfield,9.0,1.0,1.0,13.0,19.0,118.0,23.0,23.0,7.0,1.0,...,4.0,18.0,18.0,7.0,NaN,2.0,2.0,3.0,2.0,1.0


In [454]:
m = pd.read_csv('a.csv', index_col=0)
m.columns[0] = None
m.T.to_dict()


TypeError: Index does not support mutable operations

In [429]:
df.T.to_dict()

{('Sajada Bonner',): {None: 14.0,
  -1: 2.0,
  1: 2.0,
  -5: 42.0,
  'poss': 130.0,
  5: 4.0,
  -4: 10.0,
  -8: 8.0,
  -7: 24.0,
  3: 32.0,
  -9: 26.0,
  -10: 30.0,
  10: 30.0,
  18: 16.0,
  -19: 8.0,
  9: 16.0,
  13: 22.0,
  -6: 2.0,
  14: 2.0,
  -15: 38.0,
  15: 20.0,
  -16: 38.0,
  16: 20.0,
  8: 16.0,
  -3: 6.0,
  12: 6.0,
  -13: 6.0,
  -12: 12.0,
  -18: 12.0,
  4: 10.0,
  -2: 8.0,
  17: 2.0,
  11: nan,
  7: nan,
  6: nan,
  -17: nan,
  19: nan,
  -11: nan,
  2: nan,
  -14: nan},
 ('Sierra Johnson',): {None: 18.0,
  -1: 2.0,
  1: 2.0,
  -5: 28.0,
  'poss': 148.0,
  5: 14.0,
  -4: 12.0,
  -8: 14.0,
  -7: 22.0,
  3: 24.0,
  -9: 30.0,
  -10: 42.0,
  10: 42.0,
  18: 12.0,
  -19: 4.0,
  9: 28.0,
  13: 22.0,
  -6: 8.0,
  14: 2.0,
  -15: 44.0,
  15: 48.0,
  -16: 44.0,
  16: 48.0,
  8: 16.0,
  -3: 26.0,
  12: nan,
  -13: 6.0,
  -12: 12.0,
  -18: 8.0,
  4: 22.0,
  -2: 6.0,
  17: nan,
  11: 2.0,
  7: 6.0,
  6: 2.0,
  -17: 4.0,
  19: nan,
  -11: nan,
  2: nan,
  -14: nan},
 ("Ny'Ceara Pryor",

In [426]:
m['g']['poss'] += 2

In [428]:
m[('Sajada Bonner',)]['pp'] +=1

KeyError: 'pp'

In [327]:
g.boxScore['561051']

,#,Name,P,MP,GS,FGM,FGA,3FG,3FGA,FT,...,ORebs,DRebs,Tot Reb,AST,TO,STL,BLK,PF,DQ,Tech Fouls
0,23.0,Sajada Bonner,G,17:17,1,0,2,0,1,5,...,1,1,2,0,2,1,0,0,0,0
1,22.0,Ciara Brannon,F,9:24,0,0,1,0,0,0,...,1,0,1,0,2,0,0,5,1,0
2,2.0,Thai Davis,G,8:00,0,2,2,0,0,0,...,0,0,0,1,1,0,0,1,0,0
3,4.0,Nalyce Dudley,F,4:21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,24.0,Sierra Johnson,G,22:43,1,1,5,0,1,1,...,2,2,4,1,4,0,0,1,0,0
5,15.0,Emma Kirby,C,16:29,0,1,3,0,0,1,...,2,0,2,0,2,1,0,0,0,0
6,0.0,Montana Nicks,G,2:45,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,10.0,Afia Owusu-Mensah,G,8:07,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,5.0,Faith Pappas,G,12:22,0,1,2,1,2,0,...,0,0,0,0,0,0,0,0,0,0
9,1.0,Ny'Ceara Pryor,G,26:59,1,3,14,0,5,5,...,0,3,3,0,1,0,0,3,0,0


In [16]:
from game import Game
from team import Team
from utility import headers

import os.path
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from collections import defaultdict
from itertools import combinations

season = 2025
seasonStartDate = datetime.date(2023, 11, 7)
base_schedule = "https://stats.ncaa.org/season_divisions/18220/livestream_scoreboards?utf8=%E2%9C%93&season_division_id="

d1Teams = pd.read_csv("./Data/ncaa_synergy_id_names_map_2023_2024.csv").ncaaName.tolist()

allGames = pd.read_csv(f"games_{season-1}_{season}.csv", parse_dates=['date'])
#allGames['date'] = allGames['date'].dt.date

allPbp = pd.read_csv(f"pbp_{season-1}_{season}.csv")

class Day():
    def __init__(self, date, playerPossStats, teamEffDict, teamRebDict):
        self.processSchedule(date)
        self.loadGames()
        self.loadBoxScores()
        self.proccessPBP(playerPossStats)
        self.adjustRatings(teamEffDict, teamRebDict)

    def processSchedule(self, date):
        r = requests.get(base_schedule + f"&game_date={date.month}%2F{date.day}%2F{date.year}", headers=headers)
        
        if r.status_code != 200:
            raise Exception(f"Request for {date} was not successful")
        else:
            soup = BeautifulSoup(r.text)
        teamTags = soup.find_all('td', attrs={'class':'opponents_min_width'})
        gameIds = [tag['href'] for tag in soup.find_all('a', string='Box Score')]
        gameIds = [link.split('/')[2] for link in gameIds]

        cancelledGames = soup.find_all('div',attrs={'class':'table-responsive'})
        cancelledGames = [ix for ix, game in enumerate(cancelledGames) if 'Canceled' in game.text]

        names1 = []
        names2 = []
        ids1 = []
        ids2 = []

        for i, tag in enumerate(teamTags):
            name = tag.text.strip()
            
            
            if len(tag.contents) > 1:
                teamid = tag.contents[1]['href']
                teamid = teamid.split('/')[-1]
                name = "(".join(name.split(' (')[:-1]) # Get rid of record attached to team name
            else:
                teamid = None
            
            if i % 2 == 0:
                names1.append(name)
                ids1.append(teamid)
            else:
                names2.append(name)
                ids2.append(teamid)

        for cancelled in cancelledGames:
            names1.pop(cancelled)
            ids1.pop(cancelled)
            names2.pop(cancelled)
            ids2.pop(cancelled)
        
        schedule = pd.DataFrame({'teamName1':names1, 'teamId1':ids1, 'teamName2':names2, 'teamId2':ids2, 'gameId': gameIds})
        schedule['date'] = date

        self.schedule = schedule

    def loadGames(self):
        self.games = []
        for row in tqdm(self.schedule.itertuples()):
            if row.teamName1 in d1Teams and row.teamName2 in d1Teams:
                self.games.append(Game(row.gameId, source='web'))
            
    def loadBoxScores(self):
        playerBoxScores = pd.DataFrame()
        for game in tqdm(self.games):
            for teamid, box in game.boxScore.items():
                stats = box.iloc[:-2].copy()
                stats['teamId'] = teamid
                stats['opponentId'] = [team for team in game.teams if team != teamid][0]
                stats['gameId'] = game.gameId
                playerBoxScores = pd.concat([playerBoxScores, stats])
        playerBoxScores.reset_index(drop=True)
        self.playerBox = playerBoxScores

        teamBoxScores = pd.DataFrame()
        for game in tqdm(self.games):
            for teamid, box in game.boxScore.items():
                stats = box.iloc[-1].copy()
                stats['teamId'] = teamid
                stats['opponentId'] = [team for team in game.teams if team != teamid][0]
                stats['gameId'] = game.gameId
                teamBoxScores = pd.concat([teamBoxScores, stats.to_frame().T])
        #teamBoxScores = teamBoxScores.drop(columns=['#', 'P'])
        teamBoxScores = teamBoxScores.reset_index(drop=True)
        self.teamBox = teamBoxScores

    def proccessPBP(self, playerDict):
        teamPossessions = {}

        for game in tqdm(self.games):
            
            ends = np.where([item.isPossessionEnding for item in game.pbp.items])[0]+1
            possessions = np.split(game.pbp.items,ends)
            global poss
            for poss in possessions:
                # Event occurances for each team during the possession
                possStats = {team:[e.eventType if e.team==team else -1*e.eventType if e.team else None for e in poss] for team in game.teams}
                if any([ any(events) for events in possStats.values() ]):
                    for team in game.teams:
                        # Count the number of occurances for each event type while lineups where on the floor
                        for lineup in list(combinations(poss[-1].currentLineups[team],1)):
                            for pt in possStats[team]:
                                if pt and not np.isnan(pt):
                                    playerDict[lineup[0]][pt] += 1
                            playerDict[lineup[0]]['poss'] += 0.5
            
            for team in game.teams:
                teamPossessions[team] = sum([ any(e.team for e in poss) for poss in possessions ]) / 2
        self.teamPossessions = pd.DataFrame(teamPossessions.items(), columns=['teamId', 'possessions'])
        self.playerData = pd.DataFrame(dict(playerDict)).T

    def adjustRatings(self, effDict, rebDict):

        for game in tqdm(self.games):
            
            teams = game.teams

            points = [game.boxScore[team].PTS.iloc[-1] for team in teams]
            aprox_poss = [game.boxScore[team].FGA.iloc[-1] + game.boxScore[team].TO.iloc[-1]
                        + .46 * game.boxScore[team].FTA.iloc[-1] - game.boxScore[team].ORebs.iloc[-1] for team in game.teams]
            possessions = [(aprox_poss[0] + aprox_poss[1])/2]*2
            print
            self.adjustEff(list(map(int, teams)), points, possessions, effDict)

            rebs = [game.boxScore[team].ORebs.iloc[-1] for team in teams]
            chances = [game.boxScore[team].FGA.iloc[-1] - game.boxScore[team].FGM.iloc[-1] for team in game.teams]
            self.adjustReb(list(map(int, teams)), rebs, chances, rebDict)
    
    @staticmethod
    def adjustEff(teams: list, points: list, possessions: list, teams_dict, nationalAverage=0.92, tol=0.0001):
        poss1 = teams_dict[teams[0]]['poss']
        poss2 = teams_dict[teams[1]]['poss']

        weighted_off1 = teams_dict[teams[0]]['rating'][0] * (poss1[0] / (poss1[0] + possessions[0]))
        weighted_def2 = teams_dict[teams[1]]['rating'][1] * (poss2[1] / (poss2[1] + possessions[0]))

        weighted_def1 = teams_dict[teams[0]]['rating'][1] * (poss1[1] / (poss1[1] + possessions[1]))
        weighted_off2 = teams_dict[teams[1]]['rating'][0] * (poss2[0] / (poss2[0] + possessions[1]))

        new_d = teams_dict[teams[1]]['rating'][1]
        new_o = teams_dict[teams[0]]['rating'][0] # for first iteration

        # if it is a team's first game
        if poss1[0] == 0 or poss2[1] == 0:
            new_o, new_d = ((points[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1, \
                            ((points[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2
        else:    
            while True:
                old_o = new_o
                new_o = ((points[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1
                
                new_d = ((points[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2

                if abs(new_o - old_o) < tol:
                    break
        teams_dict[teams[0]]['rating'][0] = new_o
        teams_dict[teams[0]]['poss'][0] += possessions[0]
        teams_dict[teams[1]]['rating'][1] = new_d
        teams_dict[teams[1]]['poss'][1] += possessions[0]
        
        new_d = teams_dict[teams[0]]['rating'][1]
        new_o = teams_dict[teams[1]]['rating'][0] # for first iteration
        
        # if it is a team's first game
        if poss2[0] == 0 or poss1[1] == 0:
            new_o, new_d = ((points[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2, \
                        ((points[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
        else: 
            while True:
                old_o = new_o
                new_o = ((points[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2

                new_d = ((points[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
                
                if abs(new_o - old_o) < tol:
                    break
        teams_dict[teams[1]]['rating'][0] = new_o
        teams_dict[teams[1]]['poss'][0] += possessions[1]
        teams_dict[teams[0]]['rating'][1] = new_d
        teams_dict[teams[0]]['poss'][1] += possessions[1]

    @staticmethod
    def adjustReb(teams: list, rebounds: list, possessions: list, reb_dict, nationalAverage=0.30, tol=0.0001):
        poss1 = reb_dict[teams[0]]['poss']
        poss2 = reb_dict[teams[1]]['poss']

        weighted_off1 = reb_dict[teams[0]]['rating'][0] * (poss1[0] / (poss1[0] + possessions[0]))
        weighted_def2 = reb_dict[teams[1]]['rating'][1] * (poss2[1] / (poss2[1] + possessions[0]))

        weighted_def1 = reb_dict[teams[0]]['rating'][1] * (poss1[1] / (poss1[1] + possessions[1]))
        weighted_off2 = reb_dict[teams[1]]['rating'][0] * (poss2[0] / (poss2[0] + possessions[1]))

        new_d = reb_dict[teams[1]]['rating'][1]
        new_o = reb_dict[teams[0]]['rating'][0] # for first iteration

        # if it is a team's first game
        if poss1[0] == 0 or poss2[1] == 0:
            new_o, new_d = ((rebounds[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1, \
                            ((rebounds[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2
        else:    
            while True:
                old_o = new_o
                new_o = ((rebounds[0] / possessions[0]) / new_d)*nationalAverage * (possessions[0] / (poss1[0] + possessions[0])) + weighted_off1
                
                new_d = ((rebounds[0] / possessions[0]) / new_o)*nationalAverage * (possessions[0] / (poss2[1] + possessions[0])) + weighted_def2

                if abs(new_o - old_o) < tol:
                    break
        reb_dict[teams[0]]['rating'][0] = new_o
        reb_dict[teams[0]]['poss'][0] += possessions[0]
        reb_dict[teams[1]]['rating'][1] = new_d
        reb_dict[teams[1]]['poss'][1] += possessions[0]
        
        new_d = reb_dict[teams[0]]['rating'][1]
        new_o = reb_dict[teams[1]]['rating'][0] # for first iteration
        
        # if it is a team's first game
        if poss2[0] == 0 or poss1[1] == 0:
            new_o, new_d = ((rebounds[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2, \
                        ((rebounds[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
        else: 
            while True:
                old_o = new_o
                new_o = ((rebounds[1] / possessions[1]) / new_d)*nationalAverage * (possessions[1] / (poss2[0] + possessions[1])) + weighted_off2

                new_d = ((rebounds[1] / possessions[1]) / new_o)*nationalAverage * (possessions[1] / (poss1[1] + possessions[1])) + weighted_def1
                
                if abs(new_o - old_o) < tol:
                    break
        reb_dict[teams[1]]['rating'][0] = new_o
        reb_dict[teams[1]]['poss'][0] += possessions[1]
        reb_dict[teams[0]]['rating'][1] = new_d
        reb_dict[teams[0]]['poss'][1] += possessions[1]

today = datetime.date(2023, 11, 8)#datetime.date.today()
dateList = [seasonStartDate + datetime.timedelta(days=x) for x in range( (today - seasonStartDate).days +1 )]
dateList = [d for d in dateList if d not in allGames.date.unique()]


for date in dateList:
    print(date)

    teamBox = pd.read_csv("team_box_scores_2024_2025.csv")
    playerBox = pd.read_csv("player_box_scores_2024_2025.csv")
    playerPossessionData = pd.read_csv("player_possession_stats_2024_2025.csv", index_col=0)
    adjustedEfficiency = pd.read_csv("adjusted_efficiency_2024_2025.csv")
    adjustedRebounding = pd.read_csv("adjusted_rebounding_2024_2025.csv")

    #playerPossessionData.rename(columns={playerPossessionData.columns[0]:None})
    playerPossessionData.columns = ['poss'] + list(map(int, playerPossessionData.columns[1:]))
    playerPossDict = defaultdict(lambda: defaultdict(int), playerPossessionData.T.to_dict())
    adjEffDict = {team['teamId']:{'rating':[team['adjOff'],team['adjDef']], 'poss':[team['possOff'],team['possDef']]} 
                  for team in adjustedEfficiency.to_dict(orient='records')}
    adjRebDict = {team['teamId']:{'rating':[team['adjOffReb'],team['adjDefReb']], 'poss':[team['chancesOff'],team['chancesDef']]} 
                  for team in adjustedRebounding.to_dict(orient='records')}

    todaysGames = Day(date, playerPossDict, adjEffDict, adjRebDict)

    teamBox = pd.concat([teamBox, todaysGames.teamBox])
    playerBox = pd.concat([playerBox, todaysGames.playerBox])
    playerPossessionData = pd.DataFrame(dict(playerPossDict)).T
    playerPossessionData = playerPossessionData[[ 'poss', 1,   -1,   2,  -2,   3,   -3,   4,  -4,   5,   -5,   6,  -6,   7,  -7,
         8,  -8,   9,  -9,  10,  -10,  11, -11,  12,  -12,  13, -13,  14,  -14,
        15,  -15,  16,  -16,  17, -17,  18, -18, 19, -19]]

    adjustedEfficiency = pd.DataFrame(adjEffDict)
    adjustedEfficiency = adjustedEfficiency.explode(adjustedEfficiency.columns.tolist()).T
    adjustedEfficiency.reset_index(inplace=True)
    adjustedEfficiency.columns = ['teamId','adjOff','adjDef','possOff','possDef']
    adjustedRebounding = pd.DataFrame(adjRebDict)
    adjustedRebounding = adjustedRebounding.explode(adjustedRebounding.columns.tolist()).T
    adjustedRebounding.reset_index(inplace=True)
    adjustedRebounding.columns = ['teamId','adjOffReb','adjDefReb','chancesOff','chancesDef']

    teamBox.to_csv("team_box_scores_2024_2025.csv", index=False)
    playerBox.to_csv("player_box_scores_2024_2025.csv", index=False)
    playerPossessionData.to_csv("player_possession_stats_2024_2025.csv", index=True)
    adjustedEfficiency.to_csv("adjusted_efficiency_2024_2025.csv", index=False)
    adjustedRebounding.to_csv("adjusted_rebounding_2024_2025.csv", index=False)

    allGames = pd.concat([allGames, todaysGames.schedule])
    dfs = []
    for game in todaysGames.games:
        items = [list(event.data.values())[:5] + [ str(a[i]) for a in event.currentLineups.items() for i in range(2)] + [game.gameId] for event in game.pbp.items]
        (a,b), (c,d) = game.pbp.items[0].currentLineups.items()
        gameDf = pd.DataFrame(items, columns=['description', 'team', 'time', 'quarter', 'eventType', 'team1', 'lineup1', 'team2', 'lineup2', 'gameID'])
        dfs.append(gameDf)
    allPbp = pd.concat([allPbp] + dfs)
    

allGames.to_csv("games_2024_2025.csv", index=False)
allPbp.to_csv("pbp_2024_2025.csv", index=False)

2023-11-07


30it [01:29,  3.00s/it]
100%|██████████| 25/25 [00:00<00:00, 269.16it/s]
C:\Users\cfinl\AppData\Local\Temp\ipykernel_24764\1164293474.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  playerBox = pd.concat([playerBox, todaysGames.playerBox])


2023-11-08


29it [01:09,  2.41s/it]
100%|██████████| 20/20 [00:00<00:00, 256.82it/s]


In [14]:
todaysGames.schedule

,teamName1,teamId1,teamName2,teamId2,gameId,date


In [10]:
playerPossessionData

""


In [591]:
requests.get("https://stats.ncaa.org/season_divisions/1423/livestream_scoreboards?utf8=%E2%9C%93&season_division_id=")

<DatetimeArray>
['2023-11-06 00:00:00']
Length: 1, dtype: datetime64[ns]

In [523]:
allGames

,teamName1,teamId1,teamName2,teamId2,gameId,date


In [522]:
todaysGames.schedule

,teamName1,teamId1,teamName2,teamId2,gameId,date
0,New Orleans,560950,Oklahoma St.,560997,3970364,2023-11-07
1,Presbyterian,561252,Wichita St.,561219,3970358,2023-11-07
2,LIFE Pacific,None,UC San Diego,560566,3970464,2023-11-07
3,Norfolk St.,560957,Drexel,560722,3970359,2023-11-07
4,Allen,561792,Ga. Southern,560768,3970337,2023-11-07
5,St. Bonaventure,561053,Niagara,560952,3970400,2023-11-07
6,Xavier,561234,Youngstown St.,561237,3970431,2023-11-07
7,Oakland,560991,Michigan St.,560894,3954415,2023-11-07
8,UMES,560878,Virginia,561198,3970442,2023-11-07
9,Toledo,561161,James Madison,560827,3970444,2023-11-07


In [514]:
playerPossessionData = pd.read_csv("player_possession_stats_2024_2025.csv", index_col=0)
playerPossessionData.columns = ['poss'] + map(int, playerPossessionData.columns[1:]

Index(['poss', '1', '-1', '2', '-2', '3', '-3', '4', '-4', '5', '-5', '6',
       '-6', '7', '-7', '8', '-8', '9', '-9', '10', '-10', '11', '-11', '12',
       '-12', '13', '-13', '14', '-14', '15', '-15', '16', '-16', '17', '-17',
       '18', '-18'],
      dtype='object')

In [516]:
['poss'] + list(map(int, playerPossessionData.columns[1:]))

['poss',
 1,
 -1,
 2,
 -2,
 3,
 -3,
 4,
 -4,
 5,
 -5,
 6,
 -6,
 7,
 -7,
 8,
 -8,
 9,
 -9,
 10,
 -10,
 11,
 -11,
 12,
 -12,
 13,
 -13,
 14,
 -14,
 15,
 -15,
 16,
 -16,
 17,
 -17,
 18,
 -18]

In [506]:
adjustedEfficiency.teamId

0      561087
1      560694
2      560818
3      561123
4      561155
        ...  
355    560580
356    561246
357    561057
358    560675
359    561085
Name: teamId, Length: 360, dtype: int64

In [503]:
adjEffDict.keys()

dict_keys([561087, 560694, 560818, 561123, 561155, 561183, 560661, 560993, 560811, 560962, 561101, 561196, 560850, 560990, 561005, 560776, 560894, 560829, 560724, 560701, 560611, 560690, 561209, 561013, 560854, 561142, 560934, 561203, 560927, 560906, 560876, 560816, 560999, 561132, 560582, 560805, 560754, 561027, 561191, 560831, 560902, 561146, 560595, 560870, 561205, 560938, 561148, 560896, 560603, 560997, 560890, 561079, 560656, 561060, 560898, 561198, 560692, 561058, 561187, 561225, 561193, 560762, 561043, 560757, 561151, 560771, 561089, 560619, 560599, 561039, 560706, 560627, 560766, 560679, 561135, 560900, 560720, 560671, 560561, 560907, 560710, 561076, 560745, 561032, 560553, 560794, 560969, 561041, 561161, 560681, 560784, 560741, 561233, 561248, 561173, 560864, 560607, 561201, 561030, 561227, 560638, 561023, 561007, 561048, 560774, 560594, 560688, 560982, 561130, 560677, 560728, 560872, 561095, 560725, 561116, 561189, 561009, 561165, 561062, 560660, 560827, 560821, 560914, 56116

In [493]:
pd.DataFrame({'teamName1':names1, 'teamId1':ids1, 'teamName2':names2, 'teamId2':ids2, 'gameId': gameIds})


,teamName1,teamId1,teamName2,teamId2,gameId
0,Our Lady of the Lake,None,UIW,560549,3970333
1,BYU,560627,Montana St.,560912,3970332
2,St. Thomas(TX),562119,A&M-Corpus Christi,560556,3970334
3,Bucknell,560634,Penn St.,561013,3954316
4,Cleveland St.,560681,Bowling Green,560623,3970327
5,Sacred Heart,561051,Rhode Island,561039,3970328
6,Hofstra,560788,VCU,561193,3970340
7,Apprentice,None,UNCW,560932,3970336
8,Florida A&M,560749,Tennessee,561142,3970343
9,Warner,None,Jacksonville,560825,3970344


In [483]:
todaysGames.schedule

,teamName1,teamId1,teamName2,teamId2,gameId,date
0,#1 Iowa,560818,#1 South Carolina,561087,5254219,2023-11-06


In [472]:
{team['teamid']:{'rating':[team['adjOffReb'],team['adjDefReb']], 'poss':[team['chancesOff'],team['chancesDef']]} for team in adjustedEfficiency.to_dict(orient='records')}

KeyError: 'adjOffReb'

In [486]:
adjustedEfficiency.to_dict(orient='records')


[{'teamid': 561087,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560694,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560818,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 561123,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 561155,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 561183,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560661,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560993,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560811,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 560962,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 561101,
  'adjOff': 0.92,
  'adjDef': 0.92,
  'possOff': 0,
  'possDef': 0},
 {'teamid': 561196,
 

In [475]:
adjustedEfficiency.to_dict(orient='records')[0]

{'teamid': 561087, 'adjOff': 0.92, 'adjDef': 0.92, 'possOff': 0, 'possDef': 0}

In [463]:
stats.to_frame().T

,#,Name,P,MP,GS,FGM,FGA,3FG,3FGA,FT,...,AST,TO,STL,BLK,PF,DQ,Tech Fouls,teamId,opponentId,gameId
9,NaN,San Francisco,NaN,200:00,1,26,62,10,28,13,...,18,22,10,1,17,0,0,561073,560653,3970347
